# Upper layer의 long bridge 조작 - trial1 w/ custom HNSW

In [1]:
import numpy as np
import h5py
import struct
import networkx as nx
from heapq import heapify, heappop, heappush, heapreplace, nlargest, nsmallest
from operator import itemgetter
from random import random

from numpy.f2py.auxfuncs import throw_error
from sklearn.cluster import KMeans
from collections import defaultdict

class HNSW:
    # self._graphs[level][i] contains a {j: dist} dictionary,
    # where j is a neighbor of i and dist is distance

    # L2 / cosine (스칼라) 함수는 기존대로 유지하되 안정성 보강
    def l2_distance(self, a, b):
        return float(np.linalg.norm(a - b))

    def cosine_distance(self, a, b):
        na = np.linalg.norm(a) + 1e-12
        nb = np.linalg.norm(b) + 1e-12
        return 1.0 - float(np.dot(a, b) / (na * nb))

    def vectorized_distance_(self, x, ys):
        ys_arr = np.asarray(ys)
        if ys_arr.ndim == 1:
            ys_arr = ys_arr.reshape(1, -1)
        if self.distance_type == "l2":
            # squared or actual norm: 기존 스칼라가 np.linalg.norm을 사용하므로 일관성 위해 norm 사용
            return np.linalg.norm(ys_arr - x, axis=1)
        elif self.distance_type == "cosine":
            x_norm = x / (np.linalg.norm(x) + 1e-12)
            ys_norm = ys_arr / (np.linalg.norm(ys_arr, axis=1, keepdims=True) + 1e-12)
            return 1.0 - (ys_norm @ x_norm)
        else:
            # fallback: 호출 가능한 distance_func으로 루프
            return np.array([self.distance_func(x, y) for y in ys_arr])

    def __init__(self, distance_type, M=5, efConstruction=200, Mmax=None, heuristic=True):
        if distance_type == "l2":
            distance_func = self.l2_distance
        elif distance_type == "cosine":
            distance_func = self.cosine_distance
        else:
            raise TypeError('Please check your distance type!')
        self.distance_func = distance_func
        self.distance_type = distance_type  # 추가
        self.vectorized_distance = self.vectorized_distance_
        self._M = M
        self._efConstruction = efConstruction
        self._Mmax = 2 * M if Mmax is None else Mmax
        self._level_mult = 1 / np.log(M)
        self._graphs = []
        self._enter_point = None
        self.data = []
        self.visited_count = 0

        self._select = (
            self._select_heuristic if heuristic else self._select_naive)
        ##########
        self.visited_per_hop = []
        self.ann_per_hop = []
        #########

    ### Algorithm 1: INSERT
    def insert(self, q, efConstruction=None):

        if efConstruction is None:
            efConstruction = self._efConstruction

        distance = self.vectorized_distance_
        data = self.data
        graphs = self._graphs
        ep = self._enter_point
        M = self._M

        # line 4: determine level for the new element q
        l = int(-np.log(random()) * self._level_mult) + 1
        idx = len(data)
        data.append(q)

        if ep is not None:
            neg_dist = -distance(q, data[ep])
            # distance(q, data[ep])|

            # line 5-7: find the closest neighbor for levels above the insertion level
            for lc in reversed(graphs[l:]):
                neg_dist, ep = self._search_layer(q, [(neg_dist, ep)], lc, 1)[0]

            # line 8-17: insert q at the relevant levels; W is a candidate list
            layer0 = graphs[0]
            W = [(neg_dist, ep)]  ## 추가

            for lc in reversed(graphs[:l]):
                M_layer = M if lc is not layer0 else self._Mmax

                # line 9: update W with the closest nodes found in the graph
                W = self._search_layer(q, W, lc, efConstruction)  ## 변경

                # line 10: insert the best neighbors for q at this layer
                lc[idx] = layer_idx = {}
                self._select(layer_idx, W, M_layer, lc, heap=True)

                # line 11-13: insert bidirectional links to the new node
                for j, dist in layer_idx.items():
                    self._select(lc[j], (idx, dist), M_layer, lc)

        # line 18: create empty graphs for all new levels
        for _ in range(len(graphs), l):
            graphs.append({idx: {}})
            self._enter_point = idx

    ### Algorithm 5: K-NN-SEARCH
    def search(self, q, K=5, efSearch=20):
        """Find the K points closest to q."""

        distance = self.vectorized_distance_
        graphs = self._graphs
        ep = self._enter_point
        self.visited_count = 0

        if ep is None:
            raise ValueError("Empty graph")

        neg_dist = -distance(q, self.data[ep])

        # line 1-5: search from top layers down to the second level
        for lc in reversed(graphs[1:]):
            neg_dist, ep = self._search_layer(q, [(neg_dist, ep)], lc, 1)[0]

        ##########
        self.visited_per_hop = []
        self.ann_per_hop = []
        ##########

        # line 6: search with efSearch neighbors at the bottom level
        W = self._search_layer(q, [(neg_dist, ep)], graphs[0], efSearch)

        if K is not None:
            W = nlargest(K, W)
        else:
            W.sort(reverse=True)

        return [(idx, -md) for md, idx in W]

    ### Algorithm 2: SEARCH-LAYER
    def _search_layer(self, q, W, lc, ef):

        vectorized_distance = self.vectorized_distance
        data = self.data

        # Step 1: Initialize candidate list and visited set
        C = [(-neg_dist, idx) for neg_dist, idx in W]
        heapify(C)
        heapify(W)
        visited = set(idx for _, idx in W)

        # Step 4-17: Explore neighbors until candidate list is exhausted
        while C:
            dist, c = heappop(C)
            furthest = -W[0][0]
            if dist > furthest:
                break
            neighbors = [e for e in lc[c] if e not in visited]
            visited.update(neighbors)
            if neighbors:
                # data에서 한 번에 슬라이스하여 배열 생성 후 벡터화 계산
                ys = np.vstack([data[e] for e in neighbors])
                dists = vectorized_distance(q, ys)
                for e, dist in zip(neighbors, dists):
                    self.visited_count += 1
                    neg_dist = -float(dist)
                    if len(W) < ef:
                        heappush(C, (float(dist), e))
                        heappush(W, (neg_dist, e))
                        furthest = -W[0][0]
                    elif dist < furthest:
                        heappush(C, (float(dist), e))
                        heapreplace(W, (neg_dist, e))
                        furthest = -W[0][0]

            ##########
            self.visited_per_hop.append(len(visited))
            topk = nsmallest(min(ef, len(W)), ((-neg, idx) for neg, idx in W))  # (dist, id)
            self.ann_per_hop.append([idx for _, idx in topk])
            ##########

        return W

    ### Algorithm 3: SELECT-NEIGHBORS-SIMPLE
    def _select_naive(self, R, C, M, lc, heap=False):

        if not heap:
            idx, dist = C
            if len(R) < M:
                R[idx] = dist
            else:
                max_idx, max_dist = max(R.items(), key=itemgetter(1))
                if dist < max_dist:
                    del R[max_idx]
                    R[idx] = dist
            return

        else:
            C = nlargest(M, C)
            R.update({idx: -neg_dist for neg_dist, idx in C})

    def _select_heuristic(self, d, to_insert, m, g, heap=False):
        nb_dicts = [g[idx] for idx in d]

        def prioritize(idx, dist):
            return any(nd.get(idx, float('inf')) < dist for nd in nb_dicts), dist, idx

        if not heap:
            idx, dist = to_insert
            to_insert = [prioritize(idx, dist)]
        else:
            to_insert = nsmallest(m, (prioritize(idx, -mdist)
                                      for mdist, idx in to_insert))

        assert len(to_insert) > 0
        assert not any(idx in d for _, _, idx in to_insert)

        unchecked = m - len(d)
        assert 0 <= unchecked <= m
        to_insert, checked_ins = to_insert[:unchecked], to_insert[unchecked:]
        to_check = len(checked_ins)
        if to_check > 0:
            checked_del = nlargest(to_check, (prioritize(idx, dist)
                                              for idx, dist in d.items()))
        else:
            checked_del = []
        for _, dist, idx in to_insert:
            d[idx] = dist
        zipped = zip(checked_ins, checked_del)
        for (p_new, d_new, idx_new), (p_old, d_old, idx_old) in zipped:
            if (p_old, d_old) <= (p_new, d_new):
                break
            del d[idx_old]
            d[idx_new] = d_new
            assert len(d) == m

    # ======== Upper-layer graph editing helpers ========
    def max_level_map(self):
        """Return dict: node -> highest level it appears in."""
        lvl = defaultdict(int)
        for li, lc in enumerate(self._graphs):
            for i in lc.keys():
                if i not in lvl or li > lvl[i]:
                    lvl[i] = li
        return dict(lvl)

    def node_in_level(self, level, idx):
        return idx in self._graphs[level]

    def ensure_node_in_level(self, level, idx):
        if idx not in self._graphs[level]:
            self._graphs[level][idx] = {}

    def add_undirected_edge(self, level, u, v):
        """Add bidirectional edge (u,v) at given level using current distance func and M cap."""
        lc = self._graphs[level]
        # ensure nodes exist
        if u not in lc:
            lc[u] = {}
        if v not in lc:
            lc[v] = {}
        d = self.vectorized_distance_(self.data[u], self.data[v])
        # add to u
        self._select(lc[u], (v, d), self._M, lc, heap=False)
        # add to v
        self._select(lc[v], (u, d), self._M, lc, heap=False)

    def drop_undirected_edge(self, level, u, v):
        lc = self._graphs[level]
        if u in lc and v in lc[u]:
            del lc[u][v]
        if v in lc and u in lc[v]:
            del lc[v][u]

    # ======== Upper-layer robust swap helpers (M 제한을 반드시 지킬수 있도록) ========
    def _distance_idx(self, i, j):
        return self.vectorized_distance_(self.data[i], self.data[j])

    def _prune_node_to_M(self, level, u, candidate_ids):
        """Rebuild u's neighbor list at given level by pruning the candidate set down to M_layer using _select().
        Returns the set of kept neighbor ids. """
        lc = self._graphs[level]
        # ensure node exists
        if u not in lc:
            lc[u] = {}
        # decide cap per level
        M_layer = self._M if level != 0 else self._Mmax
        # build C as heap items (neg_dist, idx)
        C = []
        for v in set(candidate_ids):
            if v == u:
                continue
            d = self._distance_idx(u, v)
            C.append((-d, v))
        # prune
        R = {}
        self._select(R, C, M_layer, lc, heap=True)
        lc[u] = dict(R)
        return set(lc[u].keys())

    def apply_swaps_with_cap(self, level, drops, adds):
        """Apply batched drops/adds at a level while enforcing degree cap M per node and reciprocity.
        - drops: list of (u, v)
        - adds : list of (u, v)
        """
        lc = self._graphs[level]
        # build per-node to_drop/to_add
        to_drop = defaultdict(set)
        to_add = defaultdict(set)
        for (u, v) in drops:
            to_drop[u].add(v)
            to_drop[v].add(u)
        for (u, v) in adds:
            to_add[u].add(v)
            to_add[v].add(u)
        # compute pools and prune per node
        affected = set(list(to_drop.keys()) + list(to_add.keys()))
        # ensure nodes exist
        for u in affected:
            if u not in lc:
                lc[u] = {}
        # gather pools
        new_neighbors = {}
        for u in affected:
            cur = set(lc.get(u, {}).keys())
            pool = (cur - to_drop[u]) | to_add[u]
            kept = self._prune_node_to_M(level, u, pool)
            new_neighbors[u] = kept
        # reciprocity fix: ensure symmetry
        for u in affected:
            for v in list(new_neighbors[u]):
                if u not in self._graphs[level].get(v, {}):
                    # add back with distance
                    d = self._distance_idx(u, v)
                    self._select(self._graphs[level][v], (u, d), self._M if level != 0 else self._Mmax,
                                 self._graphs[level], heap=False)
            # remove broken reciprocals
            curv = set(self._graphs[level][u].keys())
            for v in list(curv):
                if u not in self._graphs[level].get(v, {}):
                    # remove u from v if needed
                    if v in self._graphs[level] and u in self._graphs[level][v]:
                        del self._graphs[level][v][u]

### Data Preparation

#### Asymmetric (BEIR: NFCorpus) — End‑to‑End embedding prep

In [ ]:
import os
import numpy as np
import time
import json
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

def get_device():
    if torch.cuda.is_available():
        return "cuda"
    if torch.backends.mps.is_available():
        return "mps"
    return "cpu"
device = get_device()
print(f"[asymmetric] using device: {device}")

# --- 1) Load BEIR NFCorpus (corpus & queries) ---
# NOTE: for BEIR loaders, the split names here are 'corpus' and 'queries' (not 'train')
print("[asymmetric] loading BEIR/nfcorpus (corpus, queries) ...")
corpus_ds  = load_dataset("beir/nfcorpus", "corpus")
queries_ds = load_dataset("beir/nfcorpus", "queries")
print(corpus_ds)   # should show key 'corpus'
print(queries_ds)  # should show key 'queries'

corpus_data  = corpus_ds["corpus"]     # BEIR corpus split
queries_data = queries_ds["queries"]   # BEIR queries split

# --- 2) Build texts and id maps ---
corpus_ids = []
corpus_texts = []
for doc in corpus_data:
    _id = str(doc.get("_id", ""))
    title = (doc.get("title", "") or "").strip()
    text  = (doc.get("text", "")  or "").strip()
    corpus_ids.append(_id)
    corpus_texts.append((title + " " + text).strip())

query_ids = []
query_texts = []
for q in queries_data:
    _id = str(q.get("_id", ""))
    title = (q.get("title", "") or "").strip()
    text  = (q.get("text", "")  or "").strip()
    query_ids.append(_id)
    # Asymmetric: queries are typically shorter; keep title+text but it's fine either way
    query_texts.append((title + " " + text).strip())
print(f"[asymmetric] corpus docs: {len(corpus_texts)} | queries: {len(query_texts)}")

# --- 3) Load a dual‑encoder suited model (asymmetric friendly) ---
# multi-qa-mpnet-base-dot-v1 works well for BEIR scenarios
model_name = "multi-qa-mpnet-base-dot-v1"
print(f"[asymmetric] loading SentenceTransformer('{model_name}') ...")
st_model = SentenceTransformer(model_name, device=device)

# --- 4) Encode corpus & queries ---
# Tip: larger batch sizes help on GPU; reduce on CPU
batch_corpus = 128 if device != "cpu" else 32
batch_query  = 128 if device != "cpu" else 32

t0 = time.time()
print("[asymmetric] encoding corpus ...")
corpus_embeddings = st_model.encode(
    corpus_texts,
    batch_size=batch_corpus,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # cosine 유사도 용
)
t1 = time.time(); print(f"[asymmetric] corpus done in {t1-t0:.2f}s, shape={corpus_embeddings.shape}")
print("[asymmetric] encoding queries ...")
query_embeddings = st_model.encode(
    query_texts,
    batch_size=batch_query,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True  # cosine 유사도 용
)

t2 = time.time(); print(f"[asymmetric] queries done in {t2-t1:.2f}s, shape={query_embeddings.shape}")
# --- 5) Save artifacts (npy + id maps for alignment) ---
out_dir = "./datasets/asymmetric_nfcorpus"
os.makedirs(out_dir, exist_ok=True)

corp_npy = os.path.join(out_dir, "nfcorpus_corpus_embeddings.npy")
qry_npy  = os.path.join(out_dir, "nfcorpus_query_embeddings.npy")
ids_corp = os.path.join(out_dir, "nfcorpus_corpus_ids.json")
ids_qry  = os.path.join(out_dir, "nfcorpus_query_ids.json")

train = np.load(corp_npy) if os.path.exists(corp_npy) else None
test = np.load(qry_npy) if os.path.exists(qry_npy) else None
print("[asymmetric] existing corpus embeddings:", None if train is None else train.shape)
print("[asymmetric] existing query embeddings:",  None if test is None else test.shape)

np.save(corp_npy, corpus_embeddings)
np.save(qry_npy,  query_embeddings)
with open(ids_corp, "w", encoding="utf-8") as f:
    json.dump(corpus_ids, f, ensure_ascii=False)
with open(ids_qry,  "w", encoding="utf-8") as f:
    json.dump(query_ids,  f, ensure_ascii=False)
    print(f"[asymmetric] saved:\n- {corp_npy}\n- {qry_npy}\n- {ids_corp}\n- {ids_qry}")

# --- 6) (Optional) Quick sanity: cosine similarity of a random query to top-5 docs ---
try:
    import numpy as _np
    from numpy.linalg import norm as _norm
    ridx = np.random.randint(0, len(query_embeddings))
    qv = query_embeddings[ridx]
    sims = corpus_embeddings @ qv  # both normalized
    top5 = sims.argsort()[-5:][::-1]
    print("[asymmetric] sanity check — random query top5 corpus idx:", top5.tolist())
except Exception as e:
    print("[asymmetric] sanity check skipped:", e)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[asymmetric] using device: mps
[asymmetric] loading BEIR/nfcorpus (corpus, queries) ...
DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 3633
    })
})
DatasetDict({
    queries: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 3237
    })
})
[asymmetric] corpus docs: 3633 | queries: 3237
[asymmetric] loading SentenceTransformer('multi-qa-mpnet-base-dot-v1') ...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[asymmetric] encoding corpus ...


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

#### Symmetric (GloVe) — End‑to‑End embedding prep + groundtruth

In [2]:
# Symmetric Datasets
def read_fvecs(filename):
    """Reads .fvecs binary file into np.ndarray of shape (n, d)."""
    with open(filename, 'rb') as f:
        data = f.read()
    dim = struct.unpack('i', data[:4])[0]
    vecs = np.frombuffer(data, dtype=np.float32)
    vecs = vecs.reshape(-1, dim + 1)[:, 1:]  # drop the leading 'dim'
    return vecs

def read_ivecs(filename):
    """Reads .ivecs binary file into np.ndarray of shape (n, k)."""
    with open(filename, 'rb') as f:
        data = f.read()
    dim = struct.unpack('i', data[:4])[0]
    vecs = np.frombuffer(data, dtype=np.int32)
    vecs = vecs.reshape(-1, dim + 1)[:, 1:]
    return vecs


def compute_true_neighbors(data, query, k, distance_func):
    """Compute true k-nearest neighbors for query vectors from data."""
    true_neighbors = []
    for q in query:
        dists = np.array([distance_func(q, x) for x in data])
        nn_indices = np.argsort(dists)[:k]
        true_neighbors.append(nn_indices)
    return np.array(true_neighbors)


def exact_topk(train_subset, queries, K, distance_type='l2'):
    out = np.empty((len(queries), K), dtype=np.int32)
    for i, q in enumerate(queries):
        if distance_type == 'l2':
            d = np.sum((train_subset - q) ** 2, axis=1)
        elif distance_type == 'angular' or distance_type == 'cosine':
            # L2 정규화 필요
            q_norm = q / (np.linalg.norm(q) + 1e-12)
            train_norm = train_subset / (np.linalg.norm(train_subset, axis=1, keepdims=True) + 1e-12)
            d = 1.0 - np.dot(train_norm, q_norm)
        else:
            raise ValueError("distance_type은 'l2' 또는 'angular'이어야 합니다.")
        idx = np.argpartition(d, K)[:K]
        idx = idx[np.argsort(d[idx])]
        out[i] = idx
    return out


# 데이터셋 경로 (현재 구조에 맞춰 수정)
base_path = "./datasets"
# 데이터셋 경로
file_path = base_path + "/nytimes-256-angular.hdf5"

# train = read_fvecs(f"{base_path}/sift_base.fvecs")  # 1,000,000 × 128
# print(f"Loaded train dataset with shape: {train.shape}")
# test = read_fvecs(f"{base_path}/sift_query.fvecs")  # 10,000 × 128
# print(f"Loaded test dataset with shape: {test.shape}")
# neighbors = read_ivecs(f"{base_path}/sift_groundtruth.ivecs")  # 10,000 × 100
# print(f"Loaded neighbors dataset with shape: {neighbors.shape}")


# h5py를 사용하여 파일 열기
with h5py.File(file_path, 'r') as f:
    # HDF5 파일 내의 데이터셋 키 확인 (어떤 데이터가 있는지 모를 경우 유용)
    print(f"Keys in HDF5 file: {list(f.keys())}")

    # 각 데이터셋을 numpy 배열로 불러오기
    train = np.array(f['train'])
    test = np.array(f['test'])
    neighbors = np.array(f['neighbors'])
    # distances 데이터셋이 있다면 같이 로드할 수 있습니다.
    # distances = np.array(f['distances'])

# random sample 100,000 from train
K_value = 10
seed = 42
n_target = 100_000
rng = np.random.RandomState(seed)
idx = rng.choice(train.shape[0], n_target, replace=False)
train = train[idx]
test = test[:3000]
neighbor_subset = exact_topk(train, test, max(100, K_value), distance_type='cosine')

dim = train.shape[1]
efConstruction = 100
paramM = 16
distance_method = 'cosine'

# (1) Cosine ~= Inner Product 를 위해 L2 정규화
train_norm = train.copy()

# (3-A) Naive(랜덤 삽입 순서)
rng_vis = np.random.RandomState(42)
naive_order = rng_vis.permutation(n_target)

# # (3-B) Cluster-wise(클러스터 순서로 삽입)
# kmeans = KMeans(n_clusters=10, n_init='auto', random_state=21).fit(train_norm)
# cluster_data = defaultdict(list)
# for i, lbl in enumerate(kmeans.labels_):
#     cluster_data[int(lbl)].append(i)
#
# clustered_order = []
# for c in sorted(cluster_data.keys()):
#     clustered_order.extend(cluster_data[c])

Keys in HDF5 file: ['distances', 'neighbors', 'test', 'train']


In [3]:
### Build custom HNSW (naive / cluster-wise) — prototype

def l2_normalize_inplace(X):
    n = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    X /= n


# cosine 기반이면 정규화
if distance_method == 'cosine':
    l2_normalize_inplace(train)

# (A) naive custom HNSW
naive_custom = HNSW(distance_method, M=paramM, efConstruction=efConstruction)
for i, idx0 in enumerate(naive_order):
    if i % 1000 == 0:
        print(f"[naive_custom] inserting {i}/{len(naive_order)}")
    naive_custom.insert(train[idx0])
#
# # (B) cluster-wise custom HNSW
# cluster_custom = HNSW(distance_method, M=paramM, efConstruction=efConstruction)
# for i, idx0 in enumerate(clustered_order):
#     if i % 1000 == 0:
#         print(f"[cluster_custom] inserting {i}/{len(clustered_order)}")
#     cluster_custom.insert(train[idx0])

print("done: custom HNSW builds")

[naive_custom] inserting 0/100000
[naive_custom] inserting 1000/100000
[naive_custom] inserting 2000/100000
[naive_custom] inserting 3000/100000
[naive_custom] inserting 4000/100000
[naive_custom] inserting 5000/100000
[naive_custom] inserting 6000/100000
[naive_custom] inserting 7000/100000
[naive_custom] inserting 8000/100000
[naive_custom] inserting 9000/100000
[naive_custom] inserting 10000/100000
[naive_custom] inserting 11000/100000
[naive_custom] inserting 12000/100000
[naive_custom] inserting 13000/100000
[naive_custom] inserting 14000/100000
[naive_custom] inserting 15000/100000
[naive_custom] inserting 16000/100000
[naive_custom] inserting 17000/100000
[naive_custom] inserting 18000/100000
[naive_custom] inserting 19000/100000
[naive_custom] inserting 20000/100000
[naive_custom] inserting 21000/100000
[naive_custom] inserting 22000/100000
[naive_custom] inserting 23000/100000
[naive_custom] inserting 24000/100000
[naive_custom] inserting 25000/100000
[naive_custom] inserting 

In [4]:
from typing import List, Tuple

def ulbar_single_level(hnsw: HNSW, UL: int, q_quant=0.7, C_min=2, r_min=0.10,
                       L_neigh=3, K_per_comm=3, Bdrop=10, Badd=10, jaccard_thresh=0.5,
                       retarget_entry=False, verbose=True,
                       lid_k: int = 15, lid_prop: float = 0.10, indeg_quant: float = 0.30,
                       layer0_add_only: bool = False) -> Tuple[int, int]:
    """Apply ULBAR at one level UL. Returns (#drops,#adds).
    Uses apply_swaps_with_cap() to enforce M cap and reciprocity.
    """
    lc = hnsw._graphs[UL]
    G  = nx.Graph()
    node_set = set(lc.keys())
    for u, nbrs in lc.items():
        node_set.add(u)
        for v in nbrs.keys():
            node_set.add(v)
    G.add_nodes_from(node_set)
    for u, nbrs in lc.items():
        for v in nbrs.keys():
            if u < v:
                dist = hnsw.vectorized_distance_(hnsw.data[u], hnsw.data[v])
                G.add_edge(u, v, weight=1.0/(dist+1e-12))

    if G.number_of_nodes() == 0:
        print("no nodes found for level", UL)
        return (0, 0)
    # Compute degree map once for indegree filtering
    deg_map = {u: len(nbrs) for u, nbrs in lc.items()}
    # # Retarget entry at the first (highest) level if asked
    # if retarget_entry:
    #     deg_map = dict(G.degree())
    #     new_ep = max(deg_map, key=deg_map.get)
    #     if verbose:
    #         print(f"[ULBAR L{UL}] set entry_point -> {new_ep}")
    #     hnsw._enter_point = new_ep
    # Communities (KMeans on node embeddings)
    # Heuristic for number of clusters: k = clamp(2, sqrt(|V|), 64)
    nodes_list = list(G.nodes())
    n_nodes = len(nodes_list)
    k_guess = int(np.sqrt(max(2, n_nodes)))
    k = max(1, min(128, k_guess))
    X = np.asarray([hnsw.data[i] for i in nodes_list], dtype=float)
    # Ensure normalization consistent with distance metric (cosine expects L2-normalized)
    use_l2 = (hnsw.distance_func == hnsw.l2_distance)
    if not use_l2:
        X = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)
    km = KMeans(n_clusters=k, n_init='auto' if hasattr(KMeans, "__doc__") else 10, random_state=42)
    lbl = km.fit_predict(X)
    # Build communities as sets of node ids
    comms = []
    for cid in range(k):
        members = [nodes_list[i] for i in range(n_nodes) if int(lbl[i]) == cid]
        if members:
            comms.append(set(int(x) for x in members))
    comm_id = {}
    for cid, C in enumerate(comms):
        for x in C:
            comm_id[int(x)] = cid
    print(f"[ULBAR L{UL}] detected {len(comms)} communities (KMeans, k={k})")
    # helpers
    def coverage_and_ratio(u):
        vs = list(lc.get(u, {}).keys())
        if not vs:
            return 0, 0.0
        # u의 이웃들이 몇 개의 커뮤니티에 속하는지
        cov = len({comm_id.get(v, -1) for v in vs})
        # v가 속한 커뮤니티와 u의 커뮤니티가 다른 이웃 비율 (얼마나 겹치는가?)
        ic = sum(1 for v in vs if comm_id.get(v, -1) != comm_id.get(u, -1)) / len(vs)
        return cov, ic

    use_l2 = (hnsw.distance_func == hnsw.l2_distance)

    def dist_idx(a, b):
        if use_l2:
            return float(np.linalg.norm(hnsw.data[a] - hnsw.data[b]))
        va, vb = hnsw.data[a], hnsw.data[b]
        na = np.linalg.norm(va) + 1e-12
        nb = np.linalg.norm(vb) + 1e-12
        return 1.0 - float(np.dot(va, vb) / (na * nb))

    def jaccard_lvl(a, b):
        A = set(lc.get(a, {}).keys());
        B = set(lc.get(b, {}).keys())
        if not A and not B: return 0.0
        return len(A & B) / max(len(A | B), 1)

    # Isolation
    # iso_nodes = []
    # for u in lc.keys():
    #     cov, ic = coverage_and_ratio(u)
    #     # C_min보다 더 적은 커뮤니티에 속해있는 노드 또는 inter-cluster 비율이 r_min보다 작은 노드는 isolated로 간주
    #     if cov < C_min or ic < r_min:
    #         iso_nodes.append((u, cov, ic))
    # print(f"[ULBAR L{UL}] isolated nodes detected: {len(iso_nodes)}")
    # iso_nodes.sort(key=lambda x: (x[1], x[2]))
    # long-edge tau
    edge_lens = []
    seen = set()
    for u, nbrs in lc.items():
        for v in nbrs.keys():
            a, b = (u, v) if u < v else (v, u)
            if (a, b) in seen: continue
            seen.add((a, b))
            edge_lens.append(dist_idx(a, b))
    edge_lens = np.array(edge_lens) if len(edge_lens) else np.array([0.0])
    # 전체 edge 길이 분포 중 q_quant 분위수 값을 tau로 설정 (0.7이면 상위 30% 길이)
    tau = float(np.quantile(edge_lens, q_quant)) if edge_lens.size > 0 else 0.0
    # bad edges (inter-comm & long & redundant)
    bad_edges = []
    seen.clear()
    for u, nbrs in lc.items():
        for v in nbrs.keys():
            a, b = (u, v) if u < v else (v, u)
            if (a, b) in seen: continue
            seen.add((a, b))
            if comm_id.get(a, -1) == comm_id.get(b, -1):
                continue
            d = dist_idx(a, b)
            # long edge들만 bad edge 후보로 간주
            if d < tau:
                continue
            # 이웃 노드가 유사한 경우, 즉 같은 이웃을 많이 공유하는 경우 bad edge로 간주
            if jaccard_lvl(a, b) > jaccard_thresh:
                bad_edges.append((a, b, d))
    # fallback drops if none
    if len(bad_edges) == 0:
        intra_long = []
        seen.clear()
        for u, nbrs in lc.items():
            for v in nbrs.keys():
                a, b = (u, v) if u < v else (v, u)
                if (a, b) in seen: continue
                seen.add((a, b))
                if comm_id.get(a, -1) != comm_id.get(b, -1):
                    continue
                intra_long.append((a, b, dist_idx(a, b)))
        intra_long.sort(key=lambda x: -x[2])
        bad_edges = intra_long[:Bdrop]
    # For layer 0 policy, bypass drops and only add edges to nearest communities
    if layer0_add_only or UL == 0:
        bad_edges = []  # disable drops on L0 by policy
    # hubs per community (degree)
    deg = dict(G.degree())
    per_comm = defaultdict(list)
    for v, dv in deg.items():
        per_comm[comm_id.get(v, -1)].append((dv, v))
    for c in per_comm:
        per_comm[c].sort(reverse=True)

    # medoids
    def medoid_of_comm(C):
        nodes = list(C)
        if not nodes: return None
        sub = np.asarray([hnsw.data[i] for i in nodes])
        if use_l2:
            dsum = ((sub[:, None, :] - sub[None, :, :]) ** 2).sum(axis=2).sum(axis=1)
        else:
            subn = sub / (np.linalg.norm(sub, axis=1, keepdims=True) + 1e-12)
            S = subn @ subn.T
            dsum = (1.0 - S).sum(axis=1)
        return nodes[int(np.argmin(dsum))]

    medoids = {}
    for cid, C in enumerate(comms):
        m = medoid_of_comm(C)
        if m is not None:
            medoids[cid] = m

    def nearest_comms_to_node(u, L=3):
        cu = comm_id.get(u, -1)
        pairs = []
        for cid, m in medoids.items():
            if cid == cu: continue
            pairs.append((dist_idx(u, m), cid))
        pairs.sort()
        # print(f"[ULBAR L{UL}] node {u} nearest_comms (dist, cid): {pairs[:L]}")
        return [cid for _, cid in pairs[:L]]

    def furthest_comms_to_node(u, L=3):
        cu = comm_id.get(u, -1)
        pairs = []
        for cid, m in medoids.items():
            if cid == cu: continue
            pairs.append((dist_idx(u, m), cid))
        pairs.sort(reverse=True)
        return [cid for _, cid in pairs[:L]]

    # ---- LID estimation on current level graph (k-NN over existing neighbors)
    def estimate_lid_knn_level(lc_dict, k=15):
        lid = {}
        for u, nbrs in lc_dict.items():
            neigh = list(nbrs.keys())
            if not neigh:
                lid[u] = 0.0
                continue
            kk = min(k, len(neigh))
            # distances from u to its neighbors
            ds = np.array([dist_idx(u, v) for v in neigh], dtype=float)
            # take k smallest
            part = np.argpartition(ds, kk-1)[:kk]
            dk = np.sort(ds[part])
            r_k = float(dk[-1]) + 1e-12
            s = float(np.log(r_k / (dk + 1e-12)).mean())
            lid[u] = (1.0 / max(s, 1e-12)) if s > 0 else 0.0
        return lid

    lid_vals = estimate_lid_knn_level(lc, k=lid_k)
    n_layer = max(1, len(lc))
    # # small indegree threshold (quantile)
    # indeg_arr = np.array([deg_map.get(u, 0) for u in lc.keys()], dtype=float)
    # indeg_thr = float(np.quantile(indeg_arr, indeg_quant)) if indeg_arr.size > 0 else 0.0
    # small_indeg = {u for u in lc.keys() if deg_map.get(u, 0) <= indeg_thr}

    # pick among small indegree nodes: top lid_prop by LID
    candidates = [(u, lid_vals.get(u, 0.0)) for u in lid_vals]
    candidates.sort(key=lambda t: t[1], reverse=True)
    n_pick = max(1, int(lid_prop * n_layer))
    top_by_lid_smalldeg = candidates[:n_pick]

    iso_nodes = []
    for u, lid_u in top_by_lid_smalldeg:
        cov, ic = coverage_and_ratio(u)  # keep for logging/analysis
        iso_nodes.append((u, cov, ic, lid_u, deg_map.get(u, 0)))
    if verbose:
        print(f"[ULBAR L{UL}] LID-top∩small-indegree candidates: {len(iso_nodes)} (prop={lid_prop:.3f}, k={lid_k}")
    add_props = []

    # for layer zero, only add edges to nearest communities
    if layer0_add_only or UL == 0:
        for (u, _, _, _, _) in iso_nodes:
            for cid in nearest_comms_to_node(u, L=L_neigh):
                hubs = [v for _, v in per_comm.get(cid, [])[:K_per_comm]]
                for v in hubs:
                    add_props.append((u, v, dist_idx(u, v)))
    # otherwise, add edges to furthest communities
    else:
        for (u, _, _, _, _) in iso_nodes:
            for cid in furthest_comms_to_node(u, L=L_neigh):
                hubs = [v for _, v in per_comm.get(cid, [])[:K_per_comm]]
                for v in hubs:
                    add_props.append((v, u, dist_idx(u, v)))

    # # ADD proposals
    # add_props = []
    # # for (u,_,_) in iso_nodes[:min(30,len(iso_nodes))]:
    # for (u, _, _) in iso_nodes[:len(iso_nodes)]:
    #     for cid in nearest_comms_to_node(u, L=L_neigh):
    #         hubs = [v for _, v in per_comm.get(cid, [])[:K_per_comm]]
    #         for v in hubs:
    #             add_props.append((u, v, dist_idx(u, v)))
    # build batches
    nL = max(1, len(lc))
    Badd_eff = max(Badd, int(0.005 * nL))
    Bdrop_eff = 0 if (layer0_add_only or UL == 0) else max(Bdrop, int(0.005 * nL))
    drops_batch = [(a, b) for (a, b, _) in sorted(bad_edges, key=lambda x: -x[2])[:Bdrop_eff]]
    adds_batch = [(u, v) for (u, v, _) in sorted(add_props, key=lambda x: x[2])[:Badd_eff]]
    hnsw.apply_swaps_with_cap(UL, drops_batch, adds_batch)
    if verbose:
        print(f"[ULBAR L{UL}] cap-enforced swaps: drop={len(drops_batch)}, add={len(adds_batch)}")
    return (len(drops_batch), len(adds_batch))


def ulbar_multi_levels(hnsw: HNSW, mode='top_half', q_quant=0.7, budgets=(10, 10), jaccard_thresh=0.5, C_min=2,
                       r_min=0.1, verbose=True, lid_k: int = 15, lid_prop: float = 0.10, indeg_quant: float = 0.30, layer0_add_only: bool = True):
    """Run ULBAR across multiple levels (excluding level 0).
    mode: 'all_upper' (levels max..1), 'top_half' (from max down to max//2).
    budgets: (Bdrop, Badd)
    """
    lvl_map = hnsw.max_level_map()
    if not lvl_map:
        return []
    Lmax = max(lvl_map.values())
    if mode == 'only_L0':
        target_lvls = [0]
    elif mode == 'all_upper':
        target_lvls = list(range(Lmax, 0, -1))
    elif mode  == 'upper_half':  # top_half default
        start = Lmax
        stop = max(1, Lmax // 2)
        target_lvls = list(range(start, stop - 1, -1))
    else : # all layers
        target_lvls = list(range(Lmax, -1, -1))
    if verbose:
        print(f"[ULBAR multi] target levels: {target_lvls}")
    ops = []
    for i, UL in enumerate(target_lvls):
        drops, adds = ulbar_single_level(
            hnsw, UL, q_quant=q_quant, Bdrop=budgets[0], Badd=budgets[1], C_min=C_min, r_min=r_min,
            retarget_entry=(i == 0), jaccard_thresh=jaccard_thresh, verbose=verbose,
            lid_k=lid_k, lid_prop=lid_prop, indeg_quant=indeg_quant, layer0_add_only=layer0_add_only)
        ops.append((UL, drops, adds))
    return ops


# === L1-anchored L0 bridge (reverse direction: for each L1 node, attach L0 nodes that are close, LID-high, low indegree) ===
def l1_anchor_bridge(hnsw: HNSW,
                     lid_k: int = 15,
                     indeg_quant: float = 0.30,
                     candidate_prop: float = 0.30,
                     per_l1_limit: int = 3,
                     verbose: bool = True) -> int:
    """
    For each L1 node v, add edges at L0 from v to multiple L0 nodes u that are:
      - near v (distance small at L0),
      - LID-high (sparse in local sense),
      - indegree-low (few neighbors on L0).
    Only ADDs on L0 are performed (no drops).
    Returns the number of edges added.
    """
    # Sanity: need L1 and L0 graphs
    if len(hnsw._graphs) < 2:
        if verbose: print("[L1-anchored] no level-1 graph; nothing to do.")
        return 0
    lc0 = hnsw._graphs[0]
    lc1 = hnsw._graphs[1]
    L1_nodes = set(lc1.keys())
    if not L1_nodes or not lc0:
        if verbose: print("[L1-anchored] empty level(s); nothing to do.")
        return 0

    # Distance helper consistent with index
    use_l2 = (hnsw.distance_func == hnsw.l2_distance)
    def dist_idx(a, b):
        if use_l2:
            return float(np.linalg.norm(hnsw.data[a] - hnsw.data[b]))
        va, vb = hnsw.data[a], hnsw.data[b]
        na = np.linalg.norm(va) + 1e-12
        nb = np.linalg.norm(vb) + 1e-12
        return 1.0 - float(np.dot(va, vb) / (na * nb))

    # L0 indegree map and threshold
    deg_map = {u: len(nbrs) for u, nbrs in lc0.items()}
    indeg_arr = np.array([deg_map.get(u, 0) for u in lc0.keys()], dtype=float)
    indeg_thr = float(np.quantile(indeg_arr, indeg_quant)) if indeg_arr.size > 0 else 0.0
    small_indeg = [u for u in lc0.keys() if deg_map.get(u, 0) <= indeg_thr]

    # LID estimation over L0
    def estimate_lid_knn_level(lc_dict, k=15):
        lid = {}
        for u, nbrs in lc_dict.items():
            neigh = list(nbrs.keys())
            if not neigh:
                lid[u] = 0.0
                continue
            kk = min(k, len(neigh))
            ds = np.array([dist_idx(u, v) for v in neigh], dtype=float)
            part = np.argpartition(ds, kk-1)[:kk]
            dk = np.sort(ds[part])
            r_k = float(dk[-1]) + 1e-12
            s = float(np.log(r_k / (dk + 1e-12)).mean())
            lid[u] = (1.0 / max(s, 1e-12)) if s > 0 else 0.0
        return lid
    lid_vals = estimate_lid_knn_level(lc0, k=lid_k)

    # Candidate pool on L0: small indegree, then top by LID (limit by candidate_prop for speed)
    n0 = max(1, len(lc0))
    pool = [(u, lid_vals.get(u, 0.0), deg_map.get(u, 0)) for u in small_indeg]
    pool.sort(key=lambda t: (-t[1], t[2]))  # LID desc, indegree asc
    max_pool = max(1, int(candidate_prop * n0))
    pool = pool[:max_pool]
    pool_nodes = [u for (u, _, _) in pool]

    # Build ADDs per L1 node
    adds = []
    for v in L1_nodes:
        # ensure L0 presence for v
        hnsw.ensure_node_in_level(0, v)
        forbid = set(lc0.get(v, {}).keys()) | {v}
        # distance to candidates not already connected
        cand = [u for u in pool_nodes if u not in forbid]
        if not cand:
            continue
        # Sort by distance asc, then -LID, then indegree
        scored = []
        for u in cand:
            d = dist_idx(u, v)
            scored.append((d, -lid_vals.get(u, 0.0), deg_map.get(u, 0), u))
        scored.sort(key=lambda t: (t[0], t[1], t[2]))
        # take top per_l1_limit
        take = min(per_l1_limit, len(scored))
        for i in range(take):
            _, _, _, u = scored[i]
            adds.append((v, u))  # add edge (v,u) at L0

    if not adds:
        if verbose: print("[L1-anchored] no adds generated.")
        return 0

    if verbose:
        print(f"[L1-anchored] L1_nodes={len(L1_nodes)}, pool_L0={len(pool_nodes)}, adds={len(adds)}, indeg_thr={indeg_thr:.1f}")
    # Apply only at L0 (drops=[])
    hnsw.apply_swaps_with_cap(0, [], adds)
    return len(adds)



In [5]:

def map_internal_to_orig(idxs_internal, order_map):
    return [int(order_map[i]) for i in idxs_internal]


def calculate_recall(gt, pred, k=10):
    s = set(gt[:k])
    return sum(1 for x in pred[:k] if x in s) / float(k)


def last_or_zero(seq):
    return int(seq[-1]) if isinstance(seq, (list, tuple)) and len(seq) > 0 else 0


naive_internal_to_orig = np.array(naive_order)

import pickle


def clone_hnsw(hnsw_obj: HNSW) -> HNSW:
    """Deep-clone HNSW via pickle so we can rollback-free test variants."""
    return pickle.loads(pickle.dumps(hnsw_obj, protocol=pickle.HIGHEST_PROTOCOL))


In [7]:
# NOTE: l0_l1_bridge connects L0 candidates to nearest L1 nodes (u->v).
#       l1_anchor_bridge does the reverse: for each L1 node v, attach multiple L0 nodes u (v->u) at L0.
neighbor_subset = neighbor_subset
# === Run L1-anchored L0 bridge (reverse direction: for each L1 node, attach L0 nodes) ===
efQuick = [30, 60, 100]
for ef in efQuick:
    legacy_recs = []
    legacy_visits = []
    for i in range(min(test.shape[0], 200)):
        q = test[i]
        res = naive_custom.search(q, K=K_value, efSearch=ef)
        internal_idxs = [ix for ix, _ in res]
        orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
        legacy_recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
        legacy_visits.append(last_or_zero(naive_custom.visited_per_hop))
    print(
        f"[custom-naive before layer0 repair] ef={ef}: recall@10={np.mean(legacy_recs):.4f}, visited={np.mean(legacy_visits):.1f}")

cloned_hnsw = clone_hnsw(naive_custom)
adds_cnt = l1_anchor_bridge(
    cloned_hnsw,
    lid_k=15, indeg_quant=0.20,
    candidate_prop=0.20,  # use top 30% of L0 by LID among low indegree
    per_l1_limit=3,
    verbose=True
)
print("[L1-anchored L0 bridge] adds applied:", adds_cnt)

# quick eval
for ef in [30, 60, 100]:
    recs, visits = [], []
    Nq = min(test.shape[0], 200)
    for i in range(Nq):
        q = test[i]
        if distance_method == 'cosine':
            q = q / (np.linalg.norm(q) + 1e-12)
        out = cloned_hnsw.search(q, K=K_value, efSearch=ef)
        internal_idxs = [ix for ix, _ in out]
        orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
        recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
        visits.append(last_or_zero(cloned_hnsw.visited_per_hop))
    print(f"[L1-anchored L0 bridge eval] ef={ef}: recall@10={np.mean(recs):.4f}, visited={np.mean(visits):.1f}")


[custom-naive before layer0 repair] ef=30: recall@10=0.7615, visited=776.9
[custom-naive before layer0 repair] ef=60: recall@10=0.8275, visited=1336.4
[custom-naive before layer0 repair] ef=100: recall@10=0.8560, visited=2039.8


/tmp/ipykernel_3459813/3343454452.py:351: RuntimeWarning: invalid value encountered in log
  s = float(np.log(r_k / (dk + 1e-12)).mean())


[L1-anchored] L1_nodes=6174, pool_L0=20000, adds=18522, indeg_thr=32.0
[L1-anchored L0 bridge] adds applied: 18522
[L1-anchored L0 bridge eval] ef=30: recall@10=0.7655, visited=775.7
[L1-anchored L0 bridge eval] ef=60: recall@10=0.8260, visited=1336.0
[L1-anchored L0 bridge eval] ef=100: recall@10=0.8505, visited=2021.2


In [253]:
'''
ULBAR (Upper-Layer Bridge Audit & Repair):
	•	TOP layer 그래프 생성 → 커뮤니티 분할
	•	entry point를 상층의 최대 허브로 재설정
	•	isolated node 탐지: coverage<2 또는 inter-cluster<0.25
	•	bad long edges: inter-comm ∧ 길이 quantile 상위(0.7) ∧ redundancy(Jaccard)>0.5
	•	fallback: bad_edges가 없으면 intra-comm 장거리 상위 일부 drop
	•	ADD proposals: isolated 상위 10개 × 가까운 타 커뮤니티 3개 × 허브 3개
	•	스왑 적용(drop/add): drop_undirected_edge / add_undirected_edge
'''
# 기존 recall & visited 측정
### Quick eval (custom HNSW naive_custom) — recall & visited
efQuick = [30, 60, 100]
for ef in efQuick:
    legacy_recs = []
    legacy_visits = []
    for i in range(min(test.shape[0], 200)):
        q = test[i]
        res = naive_custom.search(q, K=K_value, efSearch=ef)
        internal_idxs = [ix for ix, _ in res]
        orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
        legacy_recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
        legacy_visits.append(last_or_zero(naive_custom.visited_per_hop))
    print(
        f"[custom-naive before TOP repair] ef={ef}: recall@10={np.mean(legacy_recs):.4f}, visited={np.mean(legacy_visits):.1f}")
# print total node counts
for i in range(max(naive_custom.max_level_map().values()), 0, -1):
    nbrs = naive_custom._graphs[i]
    total_nodes = len(nbrs)
    print(f"[ULBAR before] total nodes at L{i}: {total_nodes}")

# === Run multi-layer ULBAR (top half by default) ===
# cloned_hnsw = clone_hnsw(naive_custom)
# ops = ulbar_multi_levels(
#     cloned_hnsw,
#     mode='all',
#     q_quant=0.9,
#     budgets=(10, 10000),
#     jaccard_thresh=0.9,
#     C_min=2, r_min=0.1,
#     lid_k=15, lid_prop=0.20, indeg_quant=0.30,
#     layer0_add_only=True,
#     verbose=True
# )
# print("[ULBAR multi] ops summary:", ops)

# === Run L0-only bridge  ===
cloned_hnsw = clone_hnsw(naive_custom)
ops = ulbar_multi_levels(
    cloned_hnsw,
    mode='only_L0',
    q_quant=0.9,
    budgets=(0, 500),              # no drops, only adds on L0
    jaccard_thresh=0.9,
    C_min=2, r_min=0.1,
    lid_k=15, lid_prop=0.10, indeg_quant=0.30,
    layer0_add_only=True,
    verbose=True
)
print("[ULBAR L0-only] ops summary:", ops)

# Verify whether all nodes in upper layer meets degree cap M, that is, no node has more than M neighbors
lvl_map = cloned_hnsw.max_level_map()
Lmax = max(lvl_map.values())
start = Lmax
for layer in range(start, 0, -1):
    lc_after = cloned_hnsw._graphs[layer]
    violations = []
    for u, nbrs in lc_after.items():
        deg_u = len(nbrs)
        if deg_u > cloned_hnsw._M:
            violations.append((u, deg_u))
    if violations:
        print(f"[ULBAR] degree cap violations@L{layer}: {len(violations)} (examples) ->", violations[:5])
        throw_error("M Degree cap violations detected after ULBAR!")
    else:
        print(f"[ULBAR] all nodes meet degree cap@L{layer} (M={cloned_hnsw._M})")

### Quick eval (custom HNSW naive_custom) — recall & visited (after TOP repair)
efQuick = [30, 60, 100]
for ef in efQuick:
    recs = []
    visits = []
    for i in range(min(test.shape[0], 200)):
        q = test[i]
        if distance_method == 'cosine':
            q = q / (np.linalg.norm(q) + 1e-12)
        res = cloned_hnsw.search(q, K=K_value, efSearch=ef)
        internal_idxs = [ix for ix, _ in res]
        orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
        recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
        visits.append(last_or_zero(cloned_hnsw.visited_per_hop))
    print(f"[custom-naive TOP repair] ef={ef}: recall@10={np.mean(recs):.4f}, visited={np.mean(visits):.1f}")

for L in range(max(cloned_hnsw.max_level_map().values()), 0, -1):
    def neighbor_set(lc):
        return {u: set(nbrs.keys()) for u, nbrs in lc.items()}
    before = neighbor_set(naive_custom._graphs[L])
    after = neighbor_set(cloned_hnsw._graphs[L])
    changed = sum(1 for u in before if before.get(u, set()) != after.get(u, set()))
    print(f"changed nodes@L{L}:", changed)


[custom-naive before TOP repair] ef=30: recall@10=0.5370, visited=229.2
[custom-naive before TOP repair] ef=60: recall@10=0.6510, visited=383.1
[custom-naive before TOP repair] ef=100: recall@10=0.7345, visited=575.8
[ULBAR before] total nodes at L8: 1
[ULBAR before] total nodes at L7: 4
[ULBAR before] total nodes at L6: 16
[ULBAR before] total nodes at L5: 51
[ULBAR before] total nodes at L4: 199
[ULBAR before] total nodes at L3: 754
[ULBAR before] total nodes at L2: 3039
[ULBAR before] total nodes at L1: 12336
[ULBAR multi] target levels: [8, 7, 6, 5, 4, 3, 2, 1, 0]
[ULBAR L8] detected 1 communities (KMeans, k=1)
[ULBAR L8] LID-top∩small-indegree candidates: 1 (prop=0.200, k=15
[ULBAR L8] cap-enforced swaps: drop=0, add=0
[ULBAR L7] detected 2 communities (KMeans, k=2)
[ULBAR L7] LID-top∩small-indegree candidates: 1 (prop=0.200, k=15
[ULBAR L7] cap-enforced swaps: drop=0, add=2
[ULBAR L6] detected 4 communities (KMeans, k=4)
[ULBAR L6] LID-top∩small-indegree candidates: 3 (prop=0.200

### Cross-validation-like tuning for ULBAR (no mutation of base index)

In [7]:
import itertools


def eval_index(index: HNSW, ef_list=(60, 100), max_queries=200):
    """Return dict with mean recall@10 and visited for ef in ef_list."""
    res = {}
    Nq = min(test.shape[0], max_queries)
    for ef in ef_list:
        recs = []
        visits = []
        for i in range(Nq):
            q = test[i]
            if distance_method == 'cosine':
                q = q / (np.linalg.norm(q) + 1e-12)
            out = index.search(q, K=K_value, efSearch=ef)
            internal_idxs = [ix for ix, _ in out]
            orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
            recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
            visits.append(last_or_zero(index.visited_per_hop))
        res[(ef, 'recall')] = float(np.mean(recs))
        res[(ef, 'visited')] = float(np.mean(visits))
    return res


def run_ulbar_on_clone(base_index: HNSW, mode='top_half', q_quant=0.7, budgets=(10, 10), jaccard=0.5):
    """Clone, run ULBAR on clone, return (clone, ops)."""
    idx_clone = clone_hnsw(base_index)
    ops = ulbar_multi_levels(idx_clone, mode=mode, q_quant=q_quant, budgets=budgets, jaccard_thresh=jaccard,
                             verbose=False)
    return idx_clone, ops


# 기존 recall & visited 측정
### Quick eval (custom HNSW naive_custom) — recall & visited
efQuick = [60, 100]
for ef in efQuick:
    legacy_recs = []
    legacy_visits = []
    for i in range(min(test.shape[0], 200)):
        q = test[i]
        res = naive_custom.search(q, K=K_value, efSearch=ef)
        internal_idxs = [ix for ix, _ in res]
        orig_idxs = map_internal_to_orig(internal_idxs, naive_internal_to_orig)
        legacy_recs.append(calculate_recall(neighbor_subset[i], orig_idxs, k=K_value))
        legacy_visits.append(last_or_zero(naive_custom.visited_per_hop))
    print(
        f"[custom-naive before TOP repair] ef={ef}: recall@10={np.mean(legacy_recs):.4f}, visited={np.mean(legacy_visits):.1f}")

# Parameter grid (compact)
modes = ['top_half', 'all_upper']
q_quants = [0.6, 0.7]
budgetses = [(10, 10), (12, 12)]
jaccard_threshold = [0.5, 0.6]

grids = list(itertools.product(modes, q_quants, budgetses, jaccard_threshold))

results = []
best = None
best_score = -1e9
best_index_clone = None

print(f"[ULBAR CV] trying {len(grids)} combos (won't mutate base index)")
for mode, qv, buds, jaccard_threshold in grids:
    idx_try, ops = run_ulbar_on_clone(naive_custom, mode=mode, q_quant=qv, budgets=buds, jaccard=jaccard_threshold)
    metrics = eval_index(idx_try, ef_list=(60, 100), max_queries=200)
    rec60 = metrics[(60, 'recall')];
    rec100 = metrics[(100, 'recall')]
    vis60 = metrics[(60, 'visited')];
    vis100 = metrics[(100, 'visited')]
    # score: prefer recall, penalize visited lightly
    score = 0.5 * rec60 + 0.5 * rec100 - 0.001 * ((vis60 + vis100) / 2.0)
    results.append({
        'mode': mode, 'q_quant': qv, 'budgets': buds,
        'jaccard': jaccard_threshold,
        'rec60': rec60, 'rec100': rec100,
        'vis60': vis60, 'vis100': vis100,
        'score': score,
        'ops': ops,
    })
    print(
        f"  {mode:9s} q={qv:.1f} buds={buds} jaccard={jaccard_threshold}| rec60={rec60:.4f}, rec100={rec100:.4f}, vis={((vis60 + vis100) / 2):.1f}, score={score:.5f}")
    if score > best_score:
        best_score = score
        best = (mode, qv, buds)
        best_index_clone = idx_try

# Summary
print("\n[ULBAR CV] Results summary (sorted by score):")
results_sorted = sorted(results, key=lambda r: r['score'], reverse=True)
for r in results_sorted:
    print(r)

print("\n[ULBAR CV] Best config:", best, "score=", best_score)
# best_index_clone holds the best modified copy; naive_custom remains unmodified

[custom-naive before TOP repair] ef=60: recall@10=0.7995, visited=921.6
[custom-naive before TOP repair] ef=100: recall@10=0.8470, visited=1341.7
[ULBAR CV] trying 16 combos (won't mutate base index)
  top_half  q=0.6 buds=(10, 10) jaccard=0.5| rec60=0.7995, rec100=0.8470, vis=1131.3, score=-0.30802
  top_half  q=0.6 buds=(10, 10) jaccard=0.6| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-0.30799
  top_half  q=0.6 buds=(12, 12) jaccard=0.5| rec60=0.7995, rec100=0.8470, vis=1131.3, score=-0.30802
  top_half  q=0.6 buds=(12, 12) jaccard=0.6| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-0.30799
  top_half  q=0.7 buds=(10, 10) jaccard=0.5| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-0.30799
  top_half  q=0.7 buds=(10, 10) jaccard=0.6| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-0.30799
  top_half  q=0.7 buds=(12, 12) jaccard=0.5| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-0.30799
  top_half  q=0.7 buds=(12, 12) jaccard=0.6| rec60=0.7995, rec100=0.8470, vis=1131.2, score=-